# Synthetic experiments presented in Beauce et al. 2021

In [1]:
import os
os.environ["OPENBLAS_NUM_THREADS"] = "1"
import sys

# import the iterative linear stress inversion package
import ILSI

import numpy as np
import h5py as h5
import matplotlib.pyplot as plt
from time import time as give_time

from matplotlib.colors import Normalize
from matplotlib.cm import ScalarMappable
import matplotlib.colorbar as clb
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

import mplstereonet

# set plotting parameters
import seaborn as sns
sns.set()
sns.set_style('ticks')
sns.set_palette('colorblind')
plt.rcParams['savefig.dpi'] = 200
plt.rcParams['svg.fonttype'] = 'none'

# define the color palette
_colors_ = ['C0', 'C2', 'C1', 'C4']

## Run the stress inversions

We first define the main routine that runs the stress inversion with different methods, on different levels of noise in the data. The routine can either run experiment #1 or experiment #2, as shown in the paper.
The stress inversion parameters I provide are those that I used in the paper. They make the code runs for quite a long time, and you might want to decrease n_resamplings to a much lower value (from 1000 to 50, for example).<br>

Running one experiment with the stress inversion parameters I provide takes a ~ 2h on my laptop.

In [2]:
def main(experiment=1):
    """
    experiment is 1 or 2.
    """
    t_start_experiment = give_time()
    # --------------------------------------
    #   synthetic experiment parameters
    # --------------------------------------
    # fault plane parameters
    n_earthquakes = 100
    np.random.seed(0)
    rakes = []
    if experiment == 1:
        # -------- experiment #1 ------------
        strike_mean, strike_std = 110., 10.
        dip_min, dip_max = 65., 90.
        strikes = np.random.normal(loc=strike_mean, scale=strike_std, size=n_earthquakes)
        dips = np.random.uniform(low=dip_min, high=dip_max, size=n_earthquakes)
        # stress tensor in the (north, west, upward) coordinate system
        sig1 = np.array([-1./np.sqrt(2.), -1./np.sqrt(2.), 0.])
        sig2 = np.array([0., 0., +1.])
        sig3 = np.array([-1./np.sqrt(2.), 1./np.sqrt(2.), 0.])
        R = 0.50
    elif experiment == 2:
        # -------- experiment #2 ------------
        strike_mean, strike_std = 110., 10.
        dip_min, dip_max = 20., 65.
        strikes = np.random.normal(loc=strike_mean, scale=strike_std, size=n_earthquakes)
        dips = np.random.uniform(low=dip_min, high=dip_max, size=n_earthquakes)
        # stress tensor in the (north, west, upward) coordinate system
        sig1 = np.array([-1./np.sqrt(2.), -1./np.sqrt(2.), 0.])
        sig2 = np.array([0., 0., +1.])
        sig3 = np.array([-1./np.sqrt(2.), 1./np.sqrt(2.), 0.])
        V = np.stack((sig1, sig2, sig3), axis=1)
        # rotate the eigenvectors by *angle* degrees about the sig1-axis
        # first, do the rotation in the eigenbasis
        rot = rotation_x1(angle)
        new_eig = rot.dot(np.identity(3))
        # next, go back to the (north, west, up) basis
        V = V.dot(new_eig)
        sig1 = V[:, 0]
        sig2 = V[:, 1]
        sig3 = V[:, 2]
        R = 0.70
    # -------------------
    # tension positive
    V = np.stack((sig1, sig2, sig3), axis=1)
    s1, s2, s3 = -1., 2*R-1., +1.
    S = np.diag(np.array([s1, s2, s3]) - np.sum([s1, s2, s3])/3.)
    S /= np.sqrt(np.sum(S**2))
    true_stress_tensor = np.dot(V, np.dot(S, V.T))
    # eigenvalue decomposition
    true_princ_stresses, true_princ_dir = \
            ILSI.utils_stress.stress_tensor_eigendecomposition(true_stress_tensor)
    print('True stress tensor:\n', true_stress_tensor)
    print('The directions of the principal stresses are:')
    for i, label in enumerate(['Most compressive', 'Intermediate', 'Least compressive']):
        print('{} stress: Azimuth={:.2f}, plunge={:.2f}'.
                format(label, ILSI.utils_stress.get_bearing_plunge(true_princ_dir[:, i])[0],
                       ILSI.utils_stress.get_bearing_plunge(true_princ_dir[:, i])[1]))
    for i in range(len(strikes)):
        # give fake rake, we only want the normal
        n, _ = ILSI.utils_stress.normal_slip_vectors(strikes[i], dips[i], 0.)
        traction = np.dot(true_stress_tensor, n.T) 
        normal_traction = np.sum(traction.squeeze()*n.squeeze(), axis=-1)*n.T
        shear_traction = traction - normal_traction
        shear_dir = shear_traction/np.sqrt(np.sum(shear_traction**2))
        # find the rake that will make slip in the same direction as shear
        s, d, r = ILSI.utils_stress.strike_dip_rake(n, shear_dir)
        rakes.append(r)
    rakes = np.float32(rakes)%360.
    # generate noisy focal mechanisms
    noise = [0., 3., 10.]
    strikes_1 = np.zeros((n_earthquakes, len(noise)), dtype=np.float32)
    dips_1 = np.zeros((n_earthquakes, len(noise)), dtype=np.float32)
    rakes_1 = np.zeros((n_earthquakes, len(noise)), dtype=np.float32)
    strikes_2 = np.zeros((n_earthquakes, len(noise)), dtype=np.float32)
    dips_2 = np.zeros((n_earthquakes, len(noise)), dtype=np.float32)
    rakes_2 = np.zeros((n_earthquakes, len(noise)), dtype=np.float32)
    for i in range(n_earthquakes):
        for n in range(len(noise)):
            s1 = strikes[i] + np.random.uniform(low=-noise[n], high=noise[n])
            d1 = np.random.uniform(low=max(0., dips[i]-noise[n]), high=min(90., dips[i]+noise[n]))
            r1 = rakes[i] + np.random.uniform(low=-noise[n], high=noise[n])
            strikes_1[i, n], dips_1[i, n], rakes_1[i, n] =\
                    s1, d1, r1
            s2, d2, r2 = ILSI.utils_stress.aux_plane(
                    strikes_1[i, n], dips_1[i, n], rakes_1[i, n])
            strikes_2[i, n], dips_2[i, n], rakes_2[i, n] =\
                    s2, d2, r2
    # make sure the parameters fall into the correct range
    strikes_1, rakes_1 = strikes_1%360., rakes_1%360. 
    strikes_2, rakes_2 = strikes_2%360., rakes_2%360.
    print('-------------')
    # --------------------------------------
    #  shear tractions on the planes defined by
    #  the noisy focal mechanisms
    # --------------------------------------
    shear_tractions = np.zeros((n_earthquakes, len(noise), 3), dtype=np.float32)
    slip_vectors = np.zeros((len(noise), n_earthquakes, 3), dtype=np.float32)
    for n in range(len(noise)):
        for i in range(n_earthquakes):
            n_, d_ = ILSI.utils_stress.normal_slip_vectors(
                    strikes_1[i, n], dips_1[i, n], rakes_1[i, n])
            _, _, shear_tractions[i, n, :] = ILSI.utils_stress.compute_traction(
                    true_stress_tensor, n_.reshape(1, 3))
            slip_vectors[n, i, :] = d_
    print('Normalized shear traction on true faults using the true stress tensor:')
    print(np.sqrt(np.sum(shear_tractions[:, 0, :]**2, axis=-1)))
    # --------------------------------
    #    stress tensor inversion
    # --------------------------------
    # inversion parameter
    friction_min = 0.1
    friction_max = 0.8
    friction_step = 0.05
    n_random_selections = 30
    n_stress_iter = 20
    n_averaging = 3
    n_resamplings = 1000
    #n_resamplings = 100
    ILSI_kwargs = {}
    ILSI_kwargs['max_n_iterations'] = 1000
    ILSI_kwargs['shear_update_atol'] = 1.e-7
    Tarantola_kwargs0 = {}
    inversion_output = {}
    methods = ['linear', 'failure_criterion', 'iterative',
               'iterative_failure_criterion']
    # --------------------------------
    # initialize output dictionary
    # --------------------------------
    for method in methods:
        inversion_output[method] = {}
        inversion_output[method]['stress_tensor'] =\
                np.zeros((len(noise), 3, 3), dtype=np.float32)
        inversion_output[method]['principal_stresses'] =\
                np.zeros((len(noise), 3), dtype=np.float32)
        inversion_output[method]['principal_directions'] =\
                np.zeros((len(noise), 3, 3), dtype=np.float32)
        inversion_output[method]['misfit'] = np.zeros(len(noise), dtype=np.float32)
        inversion_output[method]['boot_stress_tensor'] =\
                np.zeros((len(noise), n_resamplings, 3, 3), dtype=np.float32)
        inversion_output[method]['boot_principal_stresses'] =\
                np.zeros((len(noise), n_resamplings, 3), dtype=np.float32)
        inversion_output[method]['boot_principal_directions'] =\
                np.zeros((len(noise), n_resamplings, 3, 3), dtype=np.float32)
        inversion_output[method]['boot_misfit'] = np.zeros((len(noise), n_resamplings), dtype=np.float32)
    inversion_output['iterative_failure_criterion']['friction'] =\
            np.zeros(len(noise), dtype=np.float32)
    inversion_output['failure_criterion']['friction'] =\
            np.zeros(len(noise), dtype=np.float32)
    for experiment in ['true_fp_linear', 'true_fp_iterative']:
        inversion_output[experiment] = {}
        inversion_output[experiment]['stress_tensor'] =\
                np.zeros((3, 3), dtype=np.float32)
        inversion_output[experiment]['principal_stresses'] =\
                np.zeros(3, dtype=np.float32)
        inversion_output[experiment]['principal_directions'] =\
                np.zeros((3, 3), dtype=np.float32)
        inversion_output[experiment]['misfit'] = np.zeros(len(noise), dtype=np.float32)
        inversion_output[experiment]['boot_stress_tensor'] =\
                np.zeros((n_resamplings, 3, 3), dtype=np.float32)
        inversion_output[experiment]['boot_principal_stresses'] =\
                np.zeros((n_resamplings, 3), dtype=np.float32)
        inversion_output[experiment]['boot_principal_directions'] =\
                np.zeros((n_resamplings, 3, 3), dtype=np.float32)
        inversion_output[experiment]['boot_misfit'] = np.zeros((n_resamplings, len(noise)), dtype=np.float32)
    # -----------------------------------------
    # first, test the two inversion schemes on the true fault planes
    # ----------- whole data set
    inversion_output['true_fp_linear']['stress_tensor'],\
    inversion_output['true_fp_linear']['principal_stresses'],\
    inversion_output['true_fp_linear']['principal_directions'] =\
               ILSI.ilsi.Michael1984_inversion(
                       strikes, dips, rakes,
                       return_eigen=True, return_stats=False,
                       Tarantola_kwargs=Tarantola_kwargs0)
    inversion_output['true_fp_linear']['misfit'] =\
            np.mean(ILSI.utils_stress.mean_angular_residual(
                inversion_output['true_fp_linear']['stress_tensor'],
                strikes, dips, rakes))
    inversion_output['true_fp_iterative']['stress_tensor'], _,\
    inversion_output['true_fp_iterative']['principal_stresses'],\
    inversion_output['true_fp_iterative']['principal_directions'] =\
               ILSI.ilsi.iterative_linear_si(
                       strikes, dips, rakes,
                       return_eigen=True, return_stats=False,
                       Tarantola_kwargs=Tarantola_kwargs0, **ILSI_kwargs)
    inversion_output['true_fp_iterative']['misfit'] =\
            np.mean(ILSI.utils_stress.mean_angular_residual(
                inversion_output['true_fp_iterative']['stress_tensor'],
                strikes, dips, rakes))
    # ----------- bootstrapped data set
    for b in range(n_resamplings):
        bootstrapped_set = np.random.choice(
                np.arange(n_earthquakes), replace=True, size=n_earthquakes)
        strikes_b, dips_b, rakes_b = strikes[bootstrapped_set], dips[bootstrapped_set], rakes[bootstrapped_set]
        inversion_output['true_fp_linear']['boot_stress_tensor'][b, ...],\
        inversion_output['true_fp_linear']['boot_principal_stresses'][b, ...],\
        inversion_output['true_fp_linear']['boot_principal_directions'][b, ...] =\
                   ILSI.ilsi.Michael1984_inversion(
                           strikes_b, dips_b, rakes_b,
                           return_eigen=True, return_stats=False,
                           Tarantola_kwargs=Tarantola_kwargs0)
        inversion_output['true_fp_linear']['boot_misfit'][b, ...] =\
                np.mean(ILSI.utils_stress.mean_angular_residual(
                    inversion_output['true_fp_linear']['boot_stress_tensor'][b, ...],
                    strikes_b, dips_b, rakes_b))
        inversion_output['true_fp_iterative']['boot_stress_tensor'][b, ...], _,\
        inversion_output['true_fp_iterative']['boot_principal_stresses'][b, ...],\
        inversion_output['true_fp_iterative']['boot_principal_directions'][b, ...] =\
                   ILSI.ilsi.iterative_linear_si(
                           strikes_b, dips_b, rakes_b,
                           return_eigen=True, return_stats=False,
                           Tarantola_kwargs=Tarantola_kwargs0, **ILSI_kwargs)
        inversion_output['true_fp_iterative']['boot_misfit'][b, ...] =\
                np.mean(ILSI.utils_stress.mean_angular_residual(
                    inversion_output['true_fp_iterative']['boot_stress_tensor'][b, ...],
                    strikes_b, dips_b, rakes_b))
    # --------------------------------------------
    #   now, apply each method on the focal mechanism data set
    # --------------------------------------------
    Tarantola_kwargs = {}
    for n in range(len(noise)):
        Tarantola_kwargs['noise{:d}'.format(n)] = Tarantola_kwargs0.copy()
        print(f'Noise level {n}, linear inversion...')
        # simple, linear inversion
        inversion_output['linear']['stress_tensor'][n, ...],\
        inversion_output['linear']['principal_stresses'][n, ...],\
        inversion_output['linear']['principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_one_set(
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        n_random_selections=n_random_selections,
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        iterative_method=False)
        print(f'Noise level {n}, linear iterative inversion...')
        # linear inversion for both the stress tensor and shear stresses
        inversion_output['iterative']['stress_tensor'][n, ...],\
        inversion_output['iterative']['principal_stresses'][n, ...],\
        inversion_output['iterative']['principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_one_set(
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        n_random_selections=n_random_selections,
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        iterative_method=True)
        print(f'Noise level {n}, failure criterion inversion...')
        inversion_output['failure_criterion']['stress_tensor'][n, ...],\
        inversion_output['failure_criterion']['friction'][n],\
        inversion_output['failure_criterion']['principal_stresses'][n, ...],\
        inversion_output['failure_criterion']['principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_one_set_instability(
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        n_random_selections=n_random_selections,
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        friction_min=friction_min,
                        friction_max=friction_max,
                        friction_step=friction_step,
                        n_stress_iter=n_stress_iter,
                        n_averaging=n_averaging,
                        iterative_method=False, verbose=1)
        print(f'Noise level {n}, Linear iterative and failure '
              'criterion inversion...')
        inversion_output['iterative_failure_criterion']['stress_tensor'][n, ...],\
        inversion_output['iterative_failure_criterion']['friction'][n],\
        inversion_output['iterative_failure_criterion']['principal_stresses'][n, ...],\
        inversion_output['iterative_failure_criterion']['principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_one_set_instability(
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        n_random_selections=n_random_selections,
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        friction_min=friction_min,
                        friction_max=friction_max,
                        friction_step=friction_step,
                        n_stress_iter=n_stress_iter,
                        n_averaging=n_averaging,
                        plot=False, verbose=1)
        for method in methods:
            inversion_output[method]['misfit'][n] =\
                    np.mean(ILSI.utils_stress.mean_angular_residual(
                        inversion_output[method]['stress_tensor'][n, ...],
                        strikes, dips, rakes))
        # --------------------------------------------------------------
        #   non-parametric uncertainty estimation with bootstrap resampling
        # --------------------------------------------------------------       
        print(f'Noise level {n}, linear inversion (bootstrapping)...')
        # simple, linear inversion
        inversion_output['linear']['boot_stress_tensor'][n, ...],\
        inversion_output['linear']['boot_principal_stresses'][n, ...],\
        inversion_output['linear']['boot_principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_bootstrap(
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        n_resamplings=n_resamplings,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        iterative_method=False)
        print(f'Noise level {n}, Linear iterative inversion (bootstrapping)...')
        # linear inversion for both the stress tensor and shear stresses
        inversion_output['iterative']['boot_stress_tensor'][n, ...],\
        inversion_output['iterative']['boot_principal_stresses'][n, ...],\
        inversion_output['iterative']['boot_principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_bootstrap(
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        n_resamplings=n_resamplings,
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        iterative_method=True)
        print(f'Noise level {n}, failure criterion inversion (bootstrapping)...')
        inversion_output['failure_criterion']['boot_stress_tensor'][n, ...],\
        inversion_output['failure_criterion']['boot_principal_stresses'][n, ...],\
        inversion_output['failure_criterion']['boot_principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_bootstrap_instability(
                        inversion_output['failure_criterion']['principal_directions'][n, ...],
                        ILSI.utils_stress.R_(inversion_output['failure_criterion']['principal_stresses'][n, ...]),
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        inversion_output['failure_criterion']['friction'][n],
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        n_stress_iter=n_stress_iter,
                        n_resamplings=n_resamplings,
                        iterative_method=False, verbose=0)
        print(f'Noise level {n}, non-linear iterative and failure '
              'criterion inversion (bootstrapping)...')
        inversion_output['iterative_failure_criterion']['boot_stress_tensor'][n, ...],\
        inversion_output['iterative_failure_criterion']['boot_principal_stresses'][n, ...],\
        inversion_output['iterative_failure_criterion']['boot_principal_directions'][n, ...] =\
                ILSI.ilsi.inversion_bootstrap_instability(
                        inversion_output['iterative_failure_criterion']['principal_directions'][n, ...],
                        ILSI.utils_stress.R_(inversion_output['iterative_failure_criterion']['principal_stresses'][n, ...]),
                        strikes_1[:, n], dips_1[:, n], rakes_1[:, n],
                        inversion_output['iterative_failure_criterion']['friction'][n],
                        **ILSI_kwargs,
                        Tarantola_kwargs=Tarantola_kwargs['noise{:d}'.format(n)],
                        n_stress_iter=n_stress_iter,
                        n_resamplings=n_resamplings,
                        iterative_method=True, verbose=0)
        for method in methods:
            for b in range(n_resamplings):
                inversion_output[method]['boot_misfit'][n, b] =\
                        np.mean(ILSI.utils_stress.mean_angular_residual(
                            inversion_output[method]['boot_stress_tensor'][n, b, ...],
                            strikes, dips, rakes))
    inversion_output['strikes'] = np.stack((strikes_1, strikes_2), axis=2)
    inversion_output['dips'] = np.stack((dips_1, dips_2), axis=2)
    inversion_output['rakes'] = np.stack((rakes_1, rakes_2), axis=2)
    t_end_experiment = give_time()
    print('Done! The experiment ran in {:.2f} seconds'.
         format(t_end_experiment-t_start_experiment))
    return true_stress_tensor, inversion_output

In [3]:
def rotation_x1(theta):
    theta = theta*np.pi/180.
    R = np.array([[1., 0., 0.],
                  [0., np.cos(theta), -np.sin(theta)],
                  [0., np.sin(theta), np.cos(theta)]])
    return R

def rotation_x3(theta):
    theta = theta*np.pi/180.
    R = np.array([[np.cos(theta), -np.sin(theta), 0.],
                  [np.sin(theta), np.cos(theta), 0.],
                  [0., 0., 1.]])
    return R

def save_results(true_stress_tensor, inversion_output, filename):
    import h5py as h5
    with h5.File(filename, mode='w') as f:
        for key1 in inversion_output.keys():
            if isinstance(inversion_output[key1], dict):
                f.create_group(key1)
                for key2 in inversion_output[key1].keys():
                    f[key1].create_dataset(key2, data=inversion_output[key1][key2])
            else:
                f.create_dataset(key1, data=inversion_output[key1])
        f.create_dataset('true_stress_tensor', data=true_stress_tensor)

def read_results(filename):
    import h5py as h5
    inversion_output = {}
    with h5.File(filename, mode='r') as f:
        for key1 in f.keys():
            if isinstance(f[key1], h5.Dataset):
                inversion_output[key1] = f[key1][()]
            else:
                inversion_output[key1] = {}
                for key2 in f[key1].keys():
                    inversion_output[key1][key2] = f[key1][key2][()]
    return inversion_output['true_stress_tensor'], inversion_output

In [4]:
# choose which experiment to run
experiment = 1

In [5]:
# run the experiment
true_stress_tensor, inversion_output = main(experiment=experiment)

True stress tensor:
 [[-2.29934717e-17 -7.07106781e-01  0.00000000e+00]
 [-7.07106781e-01 -2.29934717e-17  0.00000000e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00]]
The directions of the principal stresses are:
Most compressive stress: Azimuth=315.00, plunge=0.00
Intermediate stress: Azimuth=180.00, plunge=90.00
Least compressive stress: Azimuth=45.00, plunge=0.00
-------------
Normalized shear traction on true faults using the true stress tensor:
[0.23964772 0.4744819  0.39893106 0.21057071 0.18334247 0.63505715
 0.37256086 0.5568246  0.539433   0.4741487  0.51663315 0.27391452
 0.41018313 0.52080023 0.46654266 0.48600668 0.24480642 0.5617521
 0.48926756 0.65069467 0.6915884  0.42743862 0.41094467 0.6397288
 0.10362118 0.69431996 0.52198756 0.5697588  0.31215602 0.26853737
 0.5073874  0.4773387  0.6525555  0.69463044 0.5809064  0.5055114
 0.31194273 0.33422723 0.5953321  0.58598965 0.6684326  0.6913522
 0.6395925  0.22495228 0.61068666 0.57838553 0.6693496  0.424447
 0.65914

/home/eric/miniconda3/envs/default/lib/python3.7/site-packages/ILSI/utils_stress.py:964: RuntimeWarning: invalid value encountered in arccos
  angle = np.arccos(np.sum(d.squeeze()*shear_dir.squeeze()))


Noise level 0, linear inversion...
Noise level 0, linear iterative inversion...
Noise level 0, failure criterion inversion...
-------- 1/5 ----------
Initial shape ratio: 0.06
----------
Stress tensor difference at iteration 0: 0.01786138117313385.
R=0.11, friction=0.60
Total instability: 0.96/Total differential instability: 0.18
Average angle: 4.12
Squared residuals: 1.10e-03
----------
Stress tensor difference at iteration 1: 0.00030851076007820666.
R=0.09, friction=0.60
Total instability: 0.94/Total differential instability: 0.16
Average angle: 4.26
Squared residuals: 1.18e-03
----------
Stress tensor difference at iteration 2: 0.00013025548832956702.
R=0.10, friction=0.60
Total instability: 0.94/Total differential instability: 0.17
Average angle: 4.16
Squared residuals: 1.12e-03
----------
Stress tensor difference at iteration 3: 3.158894105581567e-05.
R=0.10, friction=0.60
Total instability: 0.94/Total differential instability: 0.17
Average angle: 4.21
Squared residuals: 1.15e-03


Initial shape ratio: 0.07
----------
Stress tensor difference at iteration 0: 0.10171090811491013.
R=0.27, friction=0.60
Total instability: 0.96/Total differential instability: 0.21
Average angle: 4.43
Squared residuals: 9.34e-04
----------
Stress tensor difference at iteration 1: 0.15547147393226624.
R=0.53, friction=0.60
Total instability: 0.96/Total differential instability: 0.30
Average angle: 2.89
Squared residuals: 2.37e-04
----------
Stress tensor difference at iteration 2: 0.03650367259979248.
R=0.51, friction=0.60
Total instability: 0.94/Total differential instability: 0.40
Average angle: 0.41
Squared residuals: 7.33e-06
----------
Stress tensor difference at iteration 3: 6.633105523179417e-14.
R=0.51, friction=0.60
Total instability: 0.94/Total differential instability: 0.33
Average angle: 0.41
Squared residuals: 7.33e-06
Final results:
Stress tensor:
 [[ 0.01450657 -0.7062856   0.00711634]
 [-0.7062856  -0.02571375  0.02472749]
 [ 0.00711634  0.02472749  0.01120718]]
Shape r

Initial shape ratio: 0.09
----------
Stress tensor difference at iteration 0: 0.07392717897891998.
R=0.27, friction=0.60
Total instability: 0.96/Total differential instability: 0.21
Average angle: 5.17
Squared residuals: 1.25e-03
----------
Stress tensor difference at iteration 1: 0.1316065788269043.
R=0.50, friction=0.60
Total instability: 0.96/Total differential instability: 0.27
Average angle: 4.13
Squared residuals: 6.64e-04
----------
Stress tensor difference at iteration 2: 0.016005512326955795.
R=0.54, friction=0.60
Total instability: 0.95/Total differential instability: 0.36
Average angle: 3.05
Squared residuals: 2.20e-04
----------
Stress tensor difference at iteration 3: 5.105577419173279e-13.
R=0.54, friction=0.60
Total instability: 0.94/Total differential instability: 0.34
Average angle: 3.05
Squared residuals: 2.20e-04
Final results:
Stress tensor:
 [[ 0.01474058 -0.70372665  0.02278921]
 [-0.70372665 -0.047219    0.04998588]
 [ 0.02278921  0.04998588  0.03247841]]
Shape r

Initial shape ratio: 0.22
----------
Stress tensor difference at iteration 0: 0.04589443281292915.
R=0.35, friction=0.60
Total instability: 0.96/Total differential instability: 0.22
Average angle: 9.58
Squared residuals: 3.86e-03
----------
Stress tensor difference at iteration 1: 0.012228483334183693.
R=0.42, friction=0.60
Total instability: 0.96/Total differential instability: 0.25
Average angle: 9.36
Squared residuals: 3.31e-03
----------
Stress tensor difference at iteration 2: 0.015154442749917507.
R=0.36, friction=0.60
Total instability: 0.95/Total differential instability: 0.27
Average angle: 8.92
Squared residuals: 3.72e-03
----------
Stress tensor difference at iteration 3: 0.015199651941657066.
R=0.42, friction=0.60
Total instability: 0.96/Total differential instability: 0.24
Average angle: 9.25
Squared residuals: 3.22e-03
----------
Stress tensor difference at iteration 4: 0.015199637040495872.
R=0.36, friction=0.60
Total instability: 0.95/Total differential instability: 0.2

Initial shape ratio: 0.22
----------
Stress tensor difference at iteration 0: 0.05060926824808121.
R=0.36, friction=0.60
Total instability: 0.96/Total differential instability: 0.22
Average angle: 9.48
Squared residuals: 3.70e-03
----------
Stress tensor difference at iteration 1: 0.024589527398347855.
R=0.46, friction=0.60
Total instability: 0.96/Total differential instability: 0.25
Average angle: 9.25
Squared residuals: 3.00e-03
----------
Stress tensor difference at iteration 2: 0.027049308642745018.
R=0.36, friction=0.60
Total instability: 0.95/Total differential instability: 0.28
Average angle: 8.83
Squared residuals: 3.67e-03
----------
Stress tensor difference at iteration 3: 0.014673765748739243.
R=0.42, friction=0.60
Total instability: 0.96/Total differential instability: 0.24
Average angle: 9.36
Squared residuals: 3.31e-03
----------
Stress tensor difference at iteration 4: 0.015155146829783916.
R=0.36, friction=0.60
Total instability: 0.95/Total differential instability: 0.2

Noise level 2, Linear iterative inversion (bootstrapping)...
---------- Bootstrapping 1/100 ----------
Noise level 2, failure criterion inversion (bootstrapping)...
---------- Bootstrapping 1/100 ----------
Noise level 2, non-linear iterative and failure criterion inversion (bootstrapping)...
---------- Bootstrapping 1/100 ----------
Done! The experiment ran in 230.73 seconds


In [6]:
# save the results for future use (e.g. to play around with the plotting functions)
#save_results(true_stress_tensor, inversion_output, f'./experiment{experiment}.h5')

In [7]:
# or read results from previously run experiment
#true_stress_tensor, inversion_output = read_results(f'./experiment{experiment}.h5')

## Plot the results

Define plotting routines and use them to reproduce the figures shown in the paper.

In [8]:
# define plotting routines
def truncate_colormap(cmap, minval=0.0, maxval=1.0, n=100):
    from matplotlib.colors import LinearSegmentedColormap
    new_cmap = LinearSegmentedColormap.from_list(
        'trunc({n},{a:.2f},{b:.2f})'.format(n=cmap.name, a=minval, b=maxval),
        cmap(np.linspace(minval, maxval, n)))
    return new_cmap

def plot_inverted_stress_tensors(true_stress_tensor, inversion_output, axes=None, **kwargs):
    hist_kwargs = {}
    hist_kwargs['smoothing_sig'] = kwargs.get('smoothing_sig', 1)
    hist_kwargs['nbins'] = kwargs.get('nbins', 200)
    hist_kwargs['return_count'] = kwargs.get('return_count', True)
    hist_kwargs['confidence_intervals'] = kwargs.get('confidence_intervals', [95.])
    cmaps = ['Blues', 'Oranges', 'Greens', 'Reds']
    cmaps = [truncate_colormap(plt.get_cmap(cmap), minval=0., maxval=0.75) for cmap in cmaps]
    n_bootstraps = inversion_output['linear']['boot_principal_directions'][0, ...].shape[0]
    true_ps, true_pd = ILSI.utils_stress.stress_tensor_eigendecomposition(true_stress_tensor)
    true_R = ILSI.utils_stress.R_(true_ps)
    markers = ['o', 's', 'v']
    methods = ['linear', 'failure_criterion', 'iterative', 'iterative_failure_criterion']
    fig = plt.figure('inverted_stress_tensors', figsize=(18, 9))
    ax = fig.add_subplot(3, 4, 1, projection='stereonet')
    ax.set_title('True fault planes', pad=30)
    ax.plane(inversion_output['strikes'][:, 0, 0],
             inversion_output['dips'][:, 0, 0], color='k',
             lw=1.0)
    ax1 = fig.add_subplot(3, 4, 5, projection='stereonet')
    for i in range(3):
        az, pl = ILSI.utils_stress.get_bearing_plunge(true_pd[:, i])
        ax1.line(pl, az, marker=markers[i], markeredgecolor='k',
                 markeredgewidth=2, markerfacecolor='none',
                 markersize=20, label=r'True $\sigma_{{{:d}}}$: {:.1f}'u'\u00b0''|{:.1f}'u'\u00b0'.
                 format(i+1, az%360., pl))
    for method, cl, cmap in zip(['linear', 'iterative'], [_colors_[0], _colors_[2]], [cmaps[0], cmaps[2]]):
        exp = f'true_fp_{method}'
        R = ILSI.utils_stress.R_(inversion_output[exp]['principal_stresses'])
        for k in range(3):
            if k == 0:
                label = '{}, R={:.2f}, $\\vert{{\\Delta \\theta}}\\vert$={:.1f}'u'\u00b0'.\
                        format(method.capitalize(), R, inversion_output[exp]['misfit'])
            else:
                label = ''
            az, pl = ILSI.utils_stress.get_bearing_plunge(
                    inversion_output[exp]['principal_directions'][:, k])
            ax1.line(pl, az, marker=markers[k], markeredgecolor=cl, markerfacecolor='none',
                     markeredgewidth=2, markersize=15, label=label)
            boot_pd_stereo = np.zeros((n_bootstraps, 2), dtype=np.float32)
            for b in range(n_bootstraps):
                boot_pd_stereo[b, :] = ILSI.utils_stress.get_bearing_plunge(
                        inversion_output[exp]['boot_principal_directions'][b, :, k])
            count, lons_g, lats_g, levels = ILSI.utils_stress.get_CI_levels(
                    boot_pd_stereo[:, 0], boot_pd_stereo[:, 1], **hist_kwargs)
            ax1.contour(lons_g, lats_g, count, levels=levels,
                        linestyles=['solid', 'dashed', 'dashdot'][k],
                        vmin=0., colors=cl)
    fake_handle = [ax1.plot([], [], marker='', ls='')[0]]
    title_label = ['True stress tensor (R={:.2f}):'.format(true_R)]
    handles, labels = ax1.get_legend_handles_labels()
    plt.legend(fake_handle+handles, title_label+labels, loc='upper left', bbox_to_anchor=(-0.1, -0.15))
    axes = [ax1]
    titles = ['Noise-free', 'Low Noise', 'High Noise']
    for i, title in enumerate(titles):
        ax1 = fig.add_subplot(3, 4, 2+i, projection='stereonet')
        ax1.set_title(title, pad=30)
        ax1.plane(inversion_output['strikes'][:, i, 0],
                  inversion_output['dips'][:, i, 0], color='k',
                  lw=1.0)
        ax1.plane(inversion_output['strikes'][:, i, 1],
                  inversion_output['dips'][:, i, 1], color='dimgray',
                  lw=0.65)
        ax = fig.add_subplot(3, 4, 6+i, projection='stereonet')
        for j, method in enumerate(methods):
            R = ILSI.utils_stress.R_(inversion_output[method]['principal_stresses'][i, ...])
            for k in range(3):
                az, pl = ILSI.utils_stress.get_bearing_plunge(true_pd[:, k])
                ax.line(pl, az, marker=markers[k], markeredgecolor='k', markerfacecolor='none',
                        markeredgewidth=2, zorder=1, markersize=20)
                if k == 0:
                    label = '{}:\nR={:.2f}, $\\vert{{\\Delta \\theta}}\\vert$={:.1f}'u'\u00b0'.\
                            format(method.replace('_', ' ').capitalize(), R, inversion_output[method]['misfit'][i])
                else:
                    label = ''
                az, pl = ILSI.utils_stress.get_bearing_plunge(
                        inversion_output[method]['principal_directions'][i, :, k])
                ax.line(pl, az, marker=markers[k], markeredgecolor=_colors_[j], markerfacecolor='none',
                        markeredgewidth=2, markersize=15, label=label, zorder=2)
                boot_pd_stereo = np.zeros((n_bootstraps, 2), dtype=np.float32)
                for b in range(n_bootstraps):
                    boot_pd_stereo[b, :] = ILSI.utils_stress.get_bearing_plunge(
                            inversion_output[method]['boot_principal_directions'][i, b, :, k])
                count, lons_g, lats_g, levels = ILSI.utils_stress.get_CI_levels(
                        boot_pd_stereo[:, 0], boot_pd_stereo[:, 1], **hist_kwargs)
                ax.contour(lons_g, lats_g, count,
                           levels=levels, vmin=0.,
                           linestyles=['solid', 'dashed', 'dashdot'][k],
                           colors=_colors_[j])
        ax.legend(loc='upper left', bbox_to_anchor=(-0.1, -0.15))
        axes.append(ax)
    plt.subplots_adjust(top=0.88, bottom=0.11,
                        left=0.05, right=0.95,
                        hspace=0.4, wspace=0.4)
    return fig

def plot_shape_ratios(true_stress_tensor, inversion_output):
    true_ps, true_pd = ILSI.utils_stress.stress_tensor_eigendecomposition(true_stress_tensor)
    true_R = ILSI.utils_stress.R_(true_ps)
    fig = plt.figure('shape_ratios', figsize=(18, 9))
    titles = ['Noise-free', 'Low Noise', 'High Noise']
    methods = ['linear', 'failure_criterion', 'iterative', 'iterative_failure_criterion']
    n_bootstraps = inversion_output['linear']['boot_stress_tensor'].shape[1]
    ax = fig.add_subplot(3, 4, 1)
    for exp, cl in zip(['true_fp_linear', 'true_fp_iterative'], [_colors_[0], _colors_[2]]):
        Rs = np.zeros(n_bootstraps, dtype=np.float32)
        for b in range(n_bootstraps):
            Rs[b] = ILSI.utils_stress.R_(inversion_output[exp]['boot_principal_stresses'][b, :])
        ax.hist(Rs, range=(0., 1.), bins=20, lw=2.5, color=cl, histtype='step')
        ax.axvline(true_R, color='k')
        ax.set_xlabel('Shape Ratio')
        ax.set_ylabel('Count')
    for i, title in enumerate(titles):
        ax = fig.add_subplot(3, 4, 2+i)
        for j, method in enumerate(methods):
            R = ILSI.utils_stress.R_(inversion_output[method]['principal_stresses'][i, ...])
            Rs = np.zeros(n_bootstraps, dtype=np.float32)
            for b in range(n_bootstraps):
                Rs[b] = ILSI.utils_stress.R_(inversion_output[method]['boot_principal_stresses'][i, b, :])
            ax.hist(Rs, range=(0., 1.), bins=20, lw=2.5, color=_colors_[j], histtype='step')
            if j == 3:
                ax.hist(Rs, range=(0., 1.), bins=20, lw=2.5, color=_colors_[j], alpha=0.4)
            ax.axvline(true_R, color='k')
            ax.set_xlabel('Shape Ratio')
            ax.set_ylabel('Count')
    plt.subplots_adjust(top=0.93, bottom=0.11,
                        left=0.06, right=0.95,
                        hspace=0.25, wspace=0.4)
    return fig


def plot_shear_magnitudes(true_stress_tensor, inversion_output, axes=None):
    normals, slips = ILSI.utils_stress.normal_slip_vectors(
            inversion_output['strikes'][..., 0],
            inversion_output['dips'][..., 0],
            inversion_output['rakes'][..., 0])
    _, _, true_shear = ILSI.utils_stress.compute_traction(
            true_stress_tensor, normals[:, :, 0].T)
    _, _, true_fp_linear_shear = ILSI.utils_stress.compute_traction(
            inversion_output['true_fp_linear']['stress_tensor'], normals[:, :, 0].T)
    _, _, true_fp_iterative_shear = ILSI.utils_stress.compute_traction(
            inversion_output['true_fp_iterative']['stress_tensor'], normals[:, :, 0].T)
    fig = plt.figure('shear', figsize=(18, 9))
    ax1 = fig.add_subplot(2, 4, 1)
    ax1.scatter(np.sqrt(np.sum(true_shear**2, axis=-1)),
                np.sqrt(np.sum(true_fp_linear_shear**2, axis=-1)),
                color=_colors_[0], label='Naive')
    ax1.scatter(np.sqrt(np.sum(true_shear**2, axis=-1)),
                np.sqrt(np.sum(true_fp_iterative_shear**2, axis=-1)),
                color=_colors_[2], label='Iterative')
    ax1.set_aspect('equal')
    ax1.legend(loc='upper left')
    ax1.set_xlabel('True shear stress $\\vert \\tau \\vert$')
    ax1.set_ylabel('Predicted shear stress $\\vert \\tilde{\\tau} \\vert$')
    # -------------------
    ax2 = fig.add_subplot(2, 4, 5)
    ax2.hist(np.sqrt(np.sum(true_shear**2, axis=-1)), range=(0., 1.0), bins=20, color='k', histtype='step', lw=2.5, label='True')
    ax2.hist(np.sqrt(np.sum(true_fp_linear_shear**2, axis=-1)), range=(0., 1.0), bins=20, color=_colors_[0], alpha=0.5, label='Linear')
    ax2.hist(np.sqrt(np.sum(true_fp_iterative_shear**2, axis=-1)), range=(0., 1.0), bins=20, color=_colors_[2], alpha=0.5, label='Iterative')
    ax2.set_ylabel('Count')
    ax2.set_xlabel('Shear stress $\\vert \\tau \\vert$')
    ax2.legend(loc='best')
    # -------------------
    plt.subplots_adjust(top=0.98, bottom=0.15, left=0.10, right=0.98)
    return fig

In [9]:
# plot stress directions
%matplotlib qt
fig1 = plot_inverted_stress_tensors(true_stress_tensor, inversion_output, smoothing_sig=2)

In [10]:
# plot shape ratios
fig2 = plot_shape_ratios(true_stress_tensor, inversion_output)

In [11]:
# plot resolved shear stress magnitudes
fig3 = plot_shear_magnitudes(true_stress_tensor, inversion_output)